In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import tqdm
import time

import re
import string
from unidecode import unidecode
import emoji

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torchtext.vocab

from collections import Counter

import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

## Data loading

In [8]:
file_train = 'train_spam.csv'

data_train_all = pd.read_csv(file_train)

In [9]:
texts_train_all = data_train_all['text']
labels_all = data_train_all['text_type']
target_all = pd.Categorical(labels_all, categories=['ham', 'spam']).codes

## Preprocessing

Словарь строится на основе тренировочных дынных

In [10]:
lemmatizer = WordNetLemmatizer()

tokenizer = TweetTokenizer()

ENGLISH_STOP_WORDS = set(stopwords.words('english'))

In [13]:
def remove_hyperlink(words):
    return  re.sub(r"http\S+", "", words)

def to_lower(words):
    result = words.lower()
    return result

def remove_punctuation(words):
    # result = word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
    punctuation_pattern = r'[^\w\s]'
    return re.sub(punctuation_pattern, ' ', words)

def emoji_to_text(words):
    return emoji.demojize(words)

def remove_whitespace(words):
    result = words.strip()
    return result

def replace_newline(words):
    return words.replace('\n', '')

def remove_number(words, keyword=' NUMBER '):
    result = re.sub(r'\b\w*\d\w*\b', keyword, words)
    return result

def remove_currency(words, keyword=' CURRENCY '):
    currency_pattern = r'[£$€₹]'
    return re.sub(currency_pattern, keyword, words)

def remove_stop_words(words):
    result = [i for i in words if i not in ENGLISH_STOP_WORDS]
    return result

def word_lemmatizer(words):
    return [lemmatizer.lemmatize(s) for s in words]

def join_words(words):
    return " ".join(words)

In [14]:
def preprocess_pipeline(sentence):
    preprocess_utils = [
                        remove_hyperlink,
                        replace_newline,
                        to_lower,
                        emoji_to_text,
                        remove_currency,
                        remove_number,
                        # unidecode,   ####
                        remove_punctuation,
                        remove_whitespace,
                        tokenizer.tokenize,
                        # remove_stop_words, #####
                        word_lemmatizer,   #####
                    ]
    for func in preprocess_utils:
        sentence = func(sentence)
    return sentence

In [15]:
data_train_preprocess = [
    preprocess_pipeline(sent) for sent in texts_train_all.to_numpy()
]

In [16]:
texts_train, texts_val, y_train, y_val = train_test_split(
                                        data_train_preprocess,
                                        target_all,
                                        shuffle=True,
                                        random_state=42, 
                                        stratify=target_all)

### Строим словарь

In [24]:
def build_vocab(tokenized_sentences):
    counter = Counter()
    for sentence in tokenized_sentences:
        counter.update(sentence)
    vocab = torchtext.vocab.vocab(counter, min_freq=3, specials=('<UNK>', '<SOS>', '<EOS>', '<PAD>'))
    vocab.set_default_index(vocab["<UNK>"])
    return vocab

Vocab = build_vocab(texts_train)

VOCAB_SIZE = len(Vocab)
PAD_IDX = Vocab['<PAD>']
SOS_IDX = Vocab['<SOS>']
EOS_IDX = Vocab['<EOS>']
UNK_IDX = Vocab['<UNK>']

In [27]:
print(VOCAB_SIZE)
PAD_IDX

13071


3

In [111]:
def generate_batch(data):
    features, target = [], []
    for text, y in data:
        features.append(torch.tensor([Vocab['<SOS>']] + \
                                  Vocab.lookup_indices(text) + \
                                  [Vocab['<EOS>']],
                                  dtype=torch.long))
        target.append(torch.tensor(y, dtype=torch.long))
    features = pad_sequence(features, padding_value=Vocab['<PAD>'], batch_first=True)
    target = torch.tensor(target)
    return features, target


BATCH_SIZE = 32

train_loader = DataLoader(list(zip(texts_train, y_train)), batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
val_loader = DataLoader(list(zip(texts_val, y_val)), batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)

In [112]:
batch = next(iter(train_loader))
batch[1]

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 0])

## Model

In [128]:
torch.manual_seed(42)

DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cpu


In [138]:
class SpamWordRNN(nn.Module):
    def __init__(self, num_tokens, embed_size, hidden_size, drop_p=0.5):
        super(SpamWordRNN, self).__init__()

        self.num_tokens = num_tokens
        self.hidden_size = hidden_size
        self.emded_size = embed_size

        self.embedding = nn.Embedding(num_tokens, embed_size)
        self.rnn = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.dropout_embed = nn.Dropout(drop_p)
        self.dropout_rnn = nn.Dropout(drop_p)
        self.out = nn.Linear(hidden_size, 2)

    def forward(self, input):
        embedded = self.embedding(input)
        embedded = self.dropout_embed(embedded)
        _, hidden = self.rnn(embedded)
        hidden = self.dropout_rnn(hidden[-1])
        output = self.out(hidden)
        return output


In [139]:
def train(model, optimizer, iterator, criterion, epoch, n_show = 50):
    model.train()

    train_loss = 0
    current_loss = 0
    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        input, target = batch[0].to(DEVICE), batch[1].to(DEVICE)       
        output = model(input)
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        current_loss = loss.item()
        train_loss += current_loss

        # if i % n_show == 0:
        #     print(f'Epoch {epoch}, loss {current_loss:.3f}')
        
    return train_loss / len(iterator)

In [140]:
def evaluate(model, iterator, criterion):
    model.eval()

    epoch_loss = 0
    with torch.no_grad():
        for i, batch in tqdm.tqdm(enumerate(iterator)):
            input, target = batch[0].to(DEVICE), batch[1].to(DEVICE)

            output = model(input)

            loss = criterion(output, target)

            epoch_loss += loss.item()
    return epoch_loss / len(iterator)


def calc_metrics(model, iterator):
    model.eval()

    y_true = []
    y_pred = []
    with torch.no_grad():
        for i, (input, target) in tqdm.tqdm(enumerate(iterator)):
            y_true.extend(target.detach().cpu().tolist())
            input, target = input.to(DEVICE), target.to(DEVICE)

            output = model(input)

            target_pred = torch.argmax(output, axis=1).detach().cpu().tolist()
            y_pred.extend(target_pred)

    return (
        accuracy_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        f1_score(y_true, y_pred),
        roc_auc_score(y_true, y_pred)
    )

In [141]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def count_parameters(m: torch.nn.Module, only_trainable: bool = True):
    """
    Returns the total number of parameters used by `m` (only counting
    shared parameters once); if `only_trainable` is True, then only
    includes parameters with `requires_grad = True`
    """
    parameters = list(m.parameters())
    if only_trainable:
        parameters = [p for p in parameters if p.requires_grad]
    unique = {p.data_ptr(): p for p in parameters}.values()
    return sum(p.numel() for p in unique)

In [142]:
EMB_SIZE = 256
HID_SIZE = 256
DROPOUT = 0.5


model = SpamWordRNN(VOCAB_SIZE, EMB_SIZE, HID_SIZE, DROPOUT).to(DEVICE)
print(f'Number of parameters: {count_parameters(model)}')

Number of parameters: 3741442


In [ ]:
N_epochs = 10
n_show = 100

learning_rate = 0.001

In [ ]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [143]:
train_history = []
val_history = []

epoch = 0
epoch_history = []

accuracy_val = 0
precision_val = 0
recall_val = 0
f1_val = 0
roc_auc_val = 0
roc_auc_val_history = []

In [144]:
for epoch in range(epoch + 1, N_epochs+1):
    start_time = time.time()

    train_loss = train(model,
                       optimizer,
                       train_loader,
                       criterion,
                       epoch,
                       n_show)
    
    val_loss = evaluate(model, val_loader, criterion)

    accuracy_val,
    precision_val,
    recall_val,
    f1_val,
    roc_auc_val = calc_metrics(model, val_loader)
    roc_auc_val_history.append(roc_auc_val)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # if val_loss < best_valid_loss and val_loss < 2.3:
    #     best_valid_loss = valid_loss
    #     state = {
    #             'epoch': epoch,
    #             'state_dict': model.state_dict(),
    #             'optimizer': optimizer.state_dict()}
    #     torch.save(state, save_file)

    train_history.append(train_loss)
    val_history.append(val_loss)
    epoch_history.append(epoch)
    
    clear_output(True)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'Train loss: {train_loss:.3f}; Val. loss: {val_loss:.3f}')
    print(f'Accuracy {accuracy_val:.4f}, Precision {precision_val:.4f}, \
          Recall {recall_val:.4f}, f1 {f1_val:.4f}, ROC AUC {roc_auc_val:.4f}')
    
    plt.figure(figsize=[15, 4])

    plt.subplot(1, 2, 1)
    plt.title("Loss history")
    plt.plot(epoch_history, train_history, color='r', label='Train')
    plt.plot(epoch_history, val_history, color='b', label='Val')
    plt.legend()
    # plt.yscale('log')
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.title("ROC AUC history")
    plt.plot(epoch_history, roc_auc_val_history, color='b')
    plt.grid()
    
    plt.tight_layout()
    plt.show()


  0%|          | 0/10 [00:35<?, ?it/s]


KeyboardInterrupt: 